In [1]:
import re, os
from logerror import TestError

root_folder = "/mnt/hgfs/projlogs/SFW/sops1/SOPS/Performance Test - Performance_Test1"
root_folder = "/mnt/hgfs/projlogs/SFW/hq1/Performance Test - Performance_Test1"

test_folder = root_folder #+ '/' + location
files = [ f for f in os.listdir(test_folder) if os.path.isfile(os.path.join(test_folder,f)) and f.find('.log.1')>0 ]

# print('for ', location, 'found log files: ', files)
print('found', len(files), 'log files: ', files)

found 1 log files:  ['SOPS2UA5101XHD_0.log.1']


In [2]:

if len(files):
    FNAME = files[0]
else:
    FNAME = "LocalInjectorCOBAPPSTS03_0.log.1"
    
print('processing file: ', test_folder + '/' + FNAME)

with open(test_folder + '/' + FNAME,'r') as f:
    alllog = f.read()

errorlog = re.findall(r'(?<=\n).+?\[SEVERE\].+?id \d+?,[\w\s]+?\):.+?(?=\n)', alllog)
print('severe count:', len(errorlog))

processing file:  /mnt/hgfs/projlogs/SFW/sops1/SOPS/Performance Test - Performance_Test1/SOPS2UA5101XHD_0.log.1
severe count: 6938


In [3]:
errors = []
vusers = {}
for err in errorlog:
    time = err[:20]
    usr = re.search(r'user.(\d+?),.cycle', err)
    user = usr.group(1)
    user = str(user).rjust(4,'0')
    cyc = re.search(r'cycle.(\d+?),.step', err)
    cycle = cyc.group(1)
    stp = re.search(r'step\s+?(.+?),\s+?id', err)
    step = stp.group(1)
    errtype = re.search(r'status.(.+?)\)', err)
    error_type = errtype.group(1)
    message = err[err.find('):'):]
    new_record = TestError(time, user, cycle, step, error_type, message, test_folder)
    errors.append(new_record)
    if user in vusers.keys():
        vusers[user].append(new_record)
    else:
        vusers[user] = [new_record]

print('errors:', len(errors))


errors: 6938


In [4]:
sorted_keys = sorted(vusers.keys())
print(">> user, i, e.time, e.cycleid, e.errortype, etype, ")
print('erroring users', len(sorted_keys))
for user in sorted_keys:
    print('\nuser ' + user + ' err count:', len(vusers[user]))
    prior_cyc_index = -1
    consecutive_count = 0
    consecutivness = ''
    for i in range(len(vusers[user])-1,-1,-1):
        e = vusers[user][i]
        try:
            etype = e.validation
        except AttributeError:
            try:
                etype = e.dditem
            except AttributeError:
                etype = '!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!'

        # print('pre: cyc', e.cycleid, ' prior', prior_cyc_index, ' consecutive_count', consecutive_count, 'consecutivness', consecutivness )
        # determine consecutivness
        if prior_cyc_index == -1:
            prior_cyc_index = int(e.cycleid)
        else:
            if prior_cyc_index - int(e.cycleid) <= 1 and i != 0:
                consecutive_count += 1
                prior_cyc_index = int(e.cycleid)
            else:
                consecutivness = ';' + str(e.cycleid) + '-' + str(int(e.cycleid) + consecutive_count) + consecutivness
                consecutive_count = 0
                prior_cyc_index = int(e.cycleid)
        cause = e.determine_casuse()
    if len(consecutivness):
        print('~~', consecutivness[1:])

print(end='\n')
print('8'*80)
print("e.time, user, e.cycleid, e.errortype, e.cause")
causes = {}



for user in sorted_keys:
    e = vusers[user][0]
    print(e.time, user, e.cycleid, e.errortype.ljust(30), e.cause, len(vusers[user]), sep=' ; ', end=' ; ')
    if e.cause in ['Warning Message', 'Critical Message', 'Question Message', 'dynamic response']:
        print(e.cause_message, sep=' ; ')
    else:
        print()
    if e.cause in causes.keys():
        causes[e.cause] += 1
    else:
        causes[e.cause] = 1

print('count of erroring users:', len(sorted_keys))
for k,v in causes.items():
    print('cause', k, 'has count', v)


>> user, i, e.time, e.cycleid, e.errortype, etype, 
erroring users 13

user 0000 err count: 1586
~~ 0-0;34-1617

user 0001 err count: 1568
~~ 0-0;18-18;21-21;28-28;176-1738

user 0002 err count: 1571
~~ 0-0;22-22;30-30;163-1729

user 0006 err count: 1

user 0007 err count: 2
~~ 76-76

user 0008 err count: 342
~~ 0-0;41-41;43-43;165-165;167-168;173-173;178-179;181-181;183-192;194-514

user 0009 err count: 337
~~ 0-0;46-46;88-88;126-126;136-136;150-150;169-170;178-179;183-187;190-510

user 0010 err count: 221
~~ 0-219

user 0011 err count: 223
~~ 145-366

user 0012 err count: 137
~~ 19-20;23-26;28-30;32-32;34-35;38-39;47-48;51-52;56-56;58-58;61-63;65-66;69-69;72-72;74-74;76-76;79-80;83-84;86-87;90-92;94-191

user 0013 err count: 142
~~ 12-12;18-18;20-30;32-33;35-36;38-38;45-45;47-47;52-52;56-56;58-58;61-61;63-63;68-68;71-71;74-74;76-78;82-87;89-93;95-193

user 0014 err count: 434
~~ 0-82;82-97;99-106;108-185;187-218;220-236;238-244;246-247;249-355;357-381;383-440

user 0015 err count: 37